In [13]:
import main_module as m
import zoho_filter_module as fil
import pandas as pd
import re
import numpy as np
import datetime

%load_ext autoreload
%autoreload 2


import gspread
import pandas as pd
from fractions import Fraction

# Pastikan permision access di menu "Share" di rubah ke anyone with the link terlebih dahulu, agar bisa mengunakan cara OAuth
# Jika Ingin lebih dibatasi menggunakan cara google service account lebih disarankan

gc = gspread.oauth(
    credentials_filename= r'C:\Users\fajar.fatoni\Documents\Python\Data\Google Credentials\karma-oauth.json'
)

SHEET_ID = '1R9EDBeFw6qREb47TjQzyu6yr6IzjQ30xXNFOydg8hOI'
SHEET_NAME = 'Sheet1'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# Open the spreadsheet
spreadsheet_source = gc.open_by_key(SHEET_ID)

# Get all sheet names
sheet_names = [sheet.title for sheet in spreadsheet_source.worksheets()]
sheet_names

# get data gsheet

spreadsheet = gc.open_by_key(SHEET_ID)
worksheet = spreadsheet.worksheet(SHEET_NAME)
rows = worksheet.get_all_records()

# print(rows[:5])

# print('==============================')
# df = pd.DataFrame(rows, dtype="string")
zoho_df = pd.DataFrame(rows, dtype="string")
# df['Mobile Number'] = df['Mobile Number'].apply(Fraction)
zoho_df

# mengisi kolom <NA> dengan value string 'empty' supaya mudah mengecek data member dan booking dengan isin
zoho_df.fillna('empty', inplace=True)

# zoho_df['Departure_Date'] = zoho_df['Departure_Date'].apply(reorder_date_from_yyyy)
count_filter_awal = zoho_df.shape[0]
zoho_df

,First Name,Last Name,Email,Phone,Mobile
0,Martin,Smith,beaux21@icloud.com,447711004450,
1,Richard,Goldman,richard.goldman@talktalk.net,447766252264,
2,Kaye,Foskett,k.foskett@btinternet.com,447780612738,
3,Helen,Harris,potty37@aol.com,447950845677,
4,Ailsa,Charter,ailsa.charter@btinternet.com,447515035063,
5,Christopher,Palmer,palmercj@talk21.com,447932199795,
6,Stella,Monaghan Addy,stellaaddy@gmail.com,447907967303,
7,Charles,Allen,callen908@btinternet.com,447887640777,
8,James,Hudson,hudsjf@btinternet.com,447775848484,
9,Simon,Cosyns,simon.cosyns@gmail.com,447919116339,


In [15]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04eXpvS1NXTWF2RVE\_BOOKING\Booking By Year\FlyBuy Booking Data (BGAU & BGID & KE).xlsx'
df_booking = pd.read_excel(path, dtype='string', sheet_name='BOOKING_BGAU_BGID_KE')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


# ====================== Awal Filter Umum ====================== 

In [16]:
# data phone, mobile dam email member yang ada di master data (wash member)

m.clean_number(zoho_df, 'Phone')
m.clean_number(zoho_df, 'Mobile')
m.replace_not_valid_to_empty(zoho_df, 'Phone')
m.replace_not_valid_to_empty(zoho_df, 'Mobile')
zoho_df['Email'] = m.lowercase(zoho_df, 'Email')
m.clean_space(zoho_df, 'Email')
m.clean_not_valid_email_format(zoho_df, 'Email')

m.clean_number(df_booking, 'Phone1')
m.clean_number(df_booking, 'Mobile')
df_booking['Email'] = m.lowercase(df_booking, 'Email')
m.clean_space(df_booking, 'Email')
m.clean_not_valid_email_format(df_booking, 'Email')

m.clean_number(df_phone, 'Mobile')

df_email['Email'] = m.lowercase(df_email, 'Email')
m.clean_space(df_email, 'Email')
m.clean_not_valid_email_format(df_email, 'Email')

c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\main_module.py:104: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)
c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\main_module.py:105: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are set

'Email'

In [17]:
zoho_df.loc[
    (zoho_df['Phone'] == 'empty') & (zoho_df['Mobile'] != 'empty'), 
    'Phone'] = zoho_df['Mobile']
zoho_df.loc[zoho_df['Phone'] == zoho_df['Mobile'],'Mobile'] = 'empty'

In [ ]:
deleted_wrong_format = m.delete_email_wrong_format(zoho_df, 'Email')
display(deleted_wrong_format)
# deleted_wrong_format.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\deleted_wrong_format.xlsx', index=False)
count_deleted_wrong_format = deleted_wrong_format.shape[0]
count_after_deleted_wrong_format = zoho_df.shape[0]

,First Name,Last Name,Email,Phone,Mobile
17,,,fatn.asd,empty,empty


# ====================== Akhir Filter Umum ====================== 

In [19]:
# hitung jumlah duplicate
count_duplicate_general = zoho_df.loc[zoho_df.duplicated(subset=['Phone','Mobile','Email'])].shape[0]

zoho_df.loc[zoho_df.duplicated(subset=['Phone','Mobile','Email'])]

,First Name,Last Name,Email,Phone,Mobile


In [20]:
zoho_df.drop_duplicates(subset=['Phone','Mobile','Email'], inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = zoho_df.shape[0]

zoho_df

,First Name,Last Name,Email,Phone,Mobile
0,Martin,Smith,beaux21@icloud.com,447711004450,empty
1,Richard,Goldman,richard.goldman@talktalk.net,447766252264,empty
2,Kaye,Foskett,k.foskett@btinternet.com,447780612738,empty
3,Helen,Harris,potty37@aol.com,447950845677,empty
4,Ailsa,Charter,ailsa.charter@btinternet.com,447515035063,empty
5,Christopher,Palmer,palmercj@talk21.com,447932199795,empty
6,Stella,Monaghan Addy,stellaaddy@gmail.com,447907967303,empty
7,Charles,Allen,callen908@btinternet.com,447887640777,empty
8,James,Hudson,hudsjf@btinternet.com,447775848484,empty
9,Simon,Cosyns,simon.cosyns@gmail.com,447919116339,empty


In [21]:
# Cek leads test
test_leads = zoho_df.loc[
    
        zoho_df['Email'].str.contains('^test@', flags=re.I, regex=True) |
        zoho_df['Email'].str.contains('\+test', flags=re.I, regex=True) |
    
        zoho_df['First Name'].str.contains('^test$', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('^test$', flags=re.I, regex=True) |

        zoho_df['First Name'].str.contains('test 1', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('test 2', flags=re.I, regex=True)|
        zoho_df['First Name'].str.contains('test1', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('test2', flags=re.I, regex=True)|

        zoho_df['First Name'].str.contains(' test ', flags=re.I, regex=True) | 
        zoho_df['Last Name'].str.contains(' test ', flags=re.I, regex=True) | 
        zoho_df['Last Name'].str.contains(' test$', flags=re.I, regex=True)
    ]

count_leads_test = test_leads.shape[0]

# test_leads.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\test_leads_2.xlsx', index=False)
test_leads

,First Name,Last Name,Email,Phone,Mobile


In [22]:
zoho_df.drop(test_leads.index, inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

count_after_dedup_test = zoho_df.shape[0]

In [23]:
# dnm_leads = zoho_df.loc[(zoho_df['Do Not Mail'] == 'true') | (zoho_df['Do Not Contact'] == 'true')]
# count_dnm = dnm_leads.shape[0]

In [24]:
# zoho_df.drop(dnm_leads.index, inplace=True)
# zoho_df.reset_index(drop=True, inplace=True)

# count_after_dedup_dnm = zoho_df.shape[0]

In [25]:
# Cek Email OTA
email_OTA = m.find_OTA(zoho_df, 'Email')
count_email_OTA = email_OTA.shape[0]
# email_OTA.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\email_OTA.xlsx', index=False)

email_OTA

,First Name,Last Name,Email,Phone,Mobile


In [26]:
# delete email OTA
count_after_clean_OTA = m.delete_OTA(zoho_df, 'Email')
# zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\after_clean_OTA.xlsx', index=False)

zoho_df

,First Name,Last Name,Email,Phone,Mobile
0,Martin,Smith,beaux21@icloud.com,447711004450,empty
1,Richard,Goldman,richard.goldman@talktalk.net,447766252264,empty
2,Kaye,Foskett,k.foskett@btinternet.com,447780612738,empty
3,Helen,Harris,potty37@aol.com,447950845677,empty
4,Ailsa,Charter,ailsa.charter@btinternet.com,447515035063,empty
5,Christopher,Palmer,palmercj@talk21.com,447932199795,empty
6,Stella,Monaghan Addy,stellaaddy@gmail.com,447907967303,empty
7,Charles,Allen,callen908@btinternet.com,447887640777,empty
8,James,Hudson,hudsjf@btinternet.com,447775848484,empty
9,Simon,Cosyns,simon.cosyns@gmail.com,447919116339,empty


In [27]:
# Cek Email Karma
email_karma = m.find_karma(zoho_df, 'Email')
count_email_karma = email_karma.shape[0]
# email_OTA.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\email_OTA.xlsx', index=False)

email_karma

,First Name,Last Name,Email,Phone,Mobile


In [28]:
# delete email Karma
count_after_clean_karma = m.delete_karma(zoho_df, 'Email')
# zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\after_clean_OTA.xlsx', index=False)

zoho_df

,First Name,Last Name,Email,Phone,Mobile
0,Martin,Smith,beaux21@icloud.com,447711004450,empty
1,Richard,Goldman,richard.goldman@talktalk.net,447766252264,empty
2,Kaye,Foskett,k.foskett@btinternet.com,447780612738,empty
3,Helen,Harris,potty37@aol.com,447950845677,empty
4,Ailsa,Charter,ailsa.charter@btinternet.com,447515035063,empty
5,Christopher,Palmer,palmercj@talk21.com,447932199795,empty
6,Stella,Monaghan Addy,stellaaddy@gmail.com,447907967303,empty
7,Charles,Allen,callen908@btinternet.com,447887640777,empty
8,James,Hudson,hudsjf@btinternet.com,447775848484,empty
9,Simon,Cosyns,simon.cosyns@gmail.com,447919116339,empty


In [29]:
find_member_booking = zoho_df.copy()

find_member_booking.loc[find_member_booking['Phone'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1'
find_member_booking.loc[find_member_booking['Phone'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
find_member_booking.loc[find_member_booking['Mobile'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1' 
find_member_booking.loc[find_member_booking['Mobile'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
find_member_booking.loc[find_member_booking['Email'].isin(df_booking['Email']), 'Same Value Find'] = 'Booking-Email'

find_member_booking.loc[find_member_booking['Phone'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['Mobile'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'

find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

count_booking_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Booking', flags=re.I, regex=True)].shape[0]
count_member_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Member', flags=re.I, regex=True)].shape[0]

find_member_booking.replace('empty', '', inplace=True)
# find_member_booking.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\booking-member-dedup.csv', index=False)
# find_member_booking.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\booking-member-dedup.xlsx', index=False)

#find_member_booking.loc[find_member_booking['Same Value Find'] == 'Booking-Phone1'] 

In [30]:
# data phone, mobile dam email member yang ada di master data 
count_wash_member = zoho_df.loc[
    zoho_df['Phone'].isin(df_booking['Phone1']) | 
    zoho_df['Phone'].isin(df_booking['Mobile']) |
    zoho_df['Mobile'].isin(df_booking['Phone1']) | 
    zoho_df['Mobile'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email'])| 

    zoho_df['Phone'].isin(df_phone['Mobile']) | 
    zoho_df['Mobile'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

].shape[0]

zoho_df.drop(zoho_df.loc[
    zoho_df['Phone'].isin(df_booking['Phone1']) | 
    zoho_df['Phone'].isin(df_booking['Mobile']) |
    zoho_df['Mobile'].isin(df_booking['Phone1']) | 
    zoho_df['Mobile'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email']) |
    
    zoho_df['Phone'].isin(df_phone['Mobile']) | 
    zoho_df['Mobile'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

    ].index, inplace=True)
    
zoho_df.reset_index(drop=True, inplace=True)

count_after_wash_member = zoho_df.shape[0]


In [31]:
count_email_only = zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] == 'empty') & (zoho_df['Phone'] == 'empty'))].shape[0]
count_phone_only = zoho_df.loc[(zoho_df['Email'] == 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Phone'] != 'empty'))].shape[0]
count_email_and_phone = zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Phone'] != 'empty'))].shape[0]

count_blank = zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].shape[0]

# zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\blank_data.xlsx', index=False)

zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] == 'empty') & (zoho_df['Mobile'] == 'empty')), 'Contact Type'] = 'Email Only'
zoho_df.loc[(zoho_df['Email'] == 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty')), 'Contact Type'] = 'Phone Only'
zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty')), 'Contact Type'] = 'Email and Phone'

zoho_df.drop(zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].index, inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

count_after_wash_blank = zoho_df.shape[0]

In [32]:
arr_dedup = [count_deleted_wrong_format, count_duplicate_general, count_leads_test, count_email_OTA, count_email_karma, count_blank]
count_sum_dedup = sum(arr_dedup)
count_sum_dedup

1

In [33]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Filter Leads Awal : "+str(count_filter_awal))
print("Jumlah Format Email Salah : "+str(count_deleted_wrong_format))
print("Jumlah Setelah Clean Format Email Salah : "+str(count_after_deleted_wrong_format))
print("Jumlah Duplicate (Phone, Mobile dan Email) : "+str(count_duplicate_general))
print("Jumlah Setelah Clean Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Lead Test : "+str(count_leads_test))
print("Jumlah Setelah Clean Lead Test : "+str(count_after_dedup_test))
# print("Jumlah Lead Do Not Mail : "+str(count_dnm))
# print("Jumlah Setelah Clean Lead Do Not Mail : "+str(count_after_dedup_dnm))
print("Jumlah Email OTA : "+str(count_email_OTA))
print("Jumlah Setelah Clean Email OTA : "+str(count_after_clean_OTA))
print("Jumlah Email Karma : "+str(count_email_karma))
print("Jumlah Setelah Clean Email Karma : "+str(count_after_clean_karma))
print("Jumlah Wash Member : "+str(count_member_dedup))
print("Jumlah Wash Booking : "+str(count_booking_dedup))
print("Jumlah Setelah Wash Member : "+str(count_after_wash_member))
print("Jumlah Phone Email Blank (Setelah phone yang tidak valid dihapus) : "+str(count_blank))
print("Jumlah Setelah Wash Phone Email Blank : "+str(count_after_wash_blank))
print("==========================================================")
print("Contact Type")
print("==========================================================")
print("Jumlah Email Only : "+str(count_email_only))
print("Jumlah Phone Only : "+str(count_phone_only))
print("Jumlah Email dan Phone : "+str(count_email_and_phone))

Summary
Jumlah Filter Leads Awal : 18
Jumlah Format Email Salah : 1
Jumlah Setelah Clean Format Email Salah : 17
Jumlah Duplicate (Phone, Mobile dan Email) : 0
Jumlah Setelah Clean Duplicate : 17
Jumlah Lead Test : 0
Jumlah Setelah Clean Lead Test : 17
Jumlah Email OTA : 0
Jumlah Setelah Clean Email OTA : 17
Jumlah Email Karma : 0
Jumlah Setelah Clean Email Karma : 17
Jumlah Wash Member : 0
Jumlah Wash Booking : 0
Jumlah Setelah Wash Member : 17
Jumlah Phone Email Blank (Setelah phone yang tidak valid dihapus) : 0
Jumlah Setelah Wash Phone Email Blank : 17
Contact Type
Jumlah Email Only : 0
Jumlah Phone Only : 0
Jumlah Email dan Phone : 17


In [34]:
print("==========================================================")
print("Summary Report")
print("==========================================================")
print("Jumlah Data Awal : "+str(count_filter_awal))
print("Jumlah Wash Dedup : "+str(count_sum_dedup))
print("Jumlah Wash Member : "+str(count_member_dedup))
print("Jumlah Wash Booking : "+str(count_booking_dedup))
print("Jumlah Data Akhir : "+str(count_after_wash_blank))

Summary Report
Jumlah Data Awal : 18
Jumlah Wash Dedup : 1
Jumlah Wash Member : 0
Jumlah Wash Booking : 0
Jumlah Data Akhir : 17


In [35]:
zoho_df.replace('empty', '', inplace=True, regex=True)

zoho_df['First Name'] = zoho_df['First Name'].str.title()
zoho_df['Last Name'] = zoho_df['Last Name'].str.title()

# zoho_df.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\KLOM.csv', index=False)
# zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\newsletter-clean.xlsx', index=False)

In [36]:
zoho_df

,First Name,Last Name,Email,Phone,Mobile,Contact Type
0,Martin,Smith,beaux21@icloud.com,447711004450,,Email and Phone
1,Richard,Goldman,richard.goldman@talktalk.net,447766252264,,Email and Phone
2,Kaye,Foskett,k.foskett@btinternet.com,447780612738,,Email and Phone
3,Helen,Harris,potty37@aol.com,447950845677,,Email and Phone
4,Ailsa,Charter,ailsa.charter@btinternet.com,447515035063,,Email and Phone
5,Christopher,Palmer,palmercj@talk21.com,447932199795,,Email and Phone
6,Stella,Monaghan Addy,stellaaddy@gmail.com,447907967303,,Email and Phone
7,Charles,Allen,callen908@btinternet.com,447887640777,,Email and Phone
8,James,Hudson,hudsjf@btinternet.com,447775848484,,Email and Phone
9,Simon,Cosyns,simon.cosyns@gmail.com,447919116339,,Email and Phone


In [37]:
display(zoho_df)
# Update keseluruhan data, delete data yang lama terlebih dahulu
# clear all row before update
worksheet.clear()
worksheet.update([zoho_df.columns.values.tolist()] + zoho_df.values.tolist())

,First Name,Last Name,Email,Phone,Mobile,Contact Type
0,Martin,Smith,beaux21@icloud.com,447711004450,,Email and Phone
1,Richard,Goldman,richard.goldman@talktalk.net,447766252264,,Email and Phone
2,Kaye,Foskett,k.foskett@btinternet.com,447780612738,,Email and Phone
3,Helen,Harris,potty37@aol.com,447950845677,,Email and Phone
4,Ailsa,Charter,ailsa.charter@btinternet.com,447515035063,,Email and Phone
5,Christopher,Palmer,palmercj@talk21.com,447932199795,,Email and Phone
6,Stella,Monaghan Addy,stellaaddy@gmail.com,447907967303,,Email and Phone
7,Charles,Allen,callen908@btinternet.com,447887640777,,Email and Phone
8,James,Hudson,hudsjf@btinternet.com,447775848484,,Email and Phone
9,Simon,Cosyns,simon.cosyns@gmail.com,447919116339,,Email and Phone


{'spreadsheetId': '1R9EDBeFw6qREb47TjQzyu6yr6IzjQ30xXNFOydg8hOI',
 'updatedRange': 'Sheet1!A1:F18',
 'updatedRows': 18,
 'updatedColumns': 6,
 'updatedCells': 108}